 # Coursera_Capstone
 # Opening a New Movie Theater in Boston, MA
 

### Author: Thomas Abowd  

# 1. Importing Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!pip install geocoder
import geocoder
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# 2. Scraping Web Data and Creating DataFrame

In [2]:
# send the GET request
data = requests.get("https://theculturetrip.com/north-america/usa/massachusetts/articles/a-guide-to-bostons-23-neighborhoods/").text

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

# create an empty list to store neighborhood data
nlist = []

# appending data into list
for row in soup.findAll("h2"):
    nlist.append(row.text)
    
# creating a new DataFrame from the list
df = pd.DataFrame({"Neighborhood": nlist})
df

,Neighborhood
0,Allston
1,Back Bay
2,Bay Village
3,Beacon Hill
4,Brighton
5,Charlestown
6,Chinatown – Leather District
7,Dorchester
8,Downtown
9,East Boston


In [3]:
# number of rows in dataframe
df.shape

(23, 1)

# 3. Getting Geographical Coordinates

In [4]:
# defining a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Boston, US'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [5]:
# calling function to get coordinates then store in a new list using list comprehension
coordinates = [ get_latlng(neighborhood) for neighborhood in df["Neighborhood"].tolist() ]

In [6]:
coordinates

[[42.29304595466831, -71.0671685952437],
 [42.34999000000005, -71.08764999999994],
 [42.348168000000015, -71.06847074999999],
 [42.35842000000008, -71.06859999999995],
 [42.35213123053123, -71.12492353661709],
 [42.36776992509806, -71.05901694825978],
 [42.35251000000005, -71.06089999999995],
 [42.35134867454975, -71.05285041243937],
 [42.35829000000007, -71.05662999999998],
 [42.35141360528571, -71.05671282961363],
 [42.34355000000005, -71.10156999999998],
 [42.27486362817389, -71.119856070865],
 [42.30584990221593, -71.11909514176067],
 [42.278224926536424, -71.09608297802978],
 [42.35134867454975, -71.05285041243937],
 [42.33578000000006, -71.10980999999998],
 [42.36549000000008, -71.05296999999996],
 [42.28181311006276, -71.13711121314374],
 [42.330294267111, -71.08949882888999],
 [42.35224995700159, -71.0556905214992],
 [42.34256000000005, -71.07357999999994],
 [42.36394000000007, -71.06738999999999],
 [42.28219344914676, -71.14599477055154]]

In [7]:
# Adding Latitude and Longitude to the DataFrame
df2 = pd.DataFrame(coordinates, columns = ['Latitude', 'Longitude'])
df['Latitude'] = df2['Latitude']
df['Longitude'] = df2['Longitude']
df

,Neighborhood,Latitude,Longitude
0,Allston,42.293046,-71.067169
1,Back Bay,42.349990,-71.087650
2,Bay Village,42.348168,-71.068471
3,Beacon Hill,42.358420,-71.068600
4,Brighton,42.352131,-71.124924
5,Charlestown,42.367770,-71.059017
6,Chinatown – Leather District,42.352510,-71.060900
7,Dorchester,42.351349,-71.052850
8,Downtown,42.358290,-71.056630
9,East Boston,42.351414,-71.056713


# 4. Map of Boston with Neighborhoods Superimposed on Top

In [8]:
# getting the coordinates of Boston
address = 'Boston, MA'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Boston, MA {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Boston, MA 42.3602534, -71.0582912.


In [9]:
# creating a map of Boston using latitude and longitude values
map_Boston = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Boston)  
    
map_Boston

# 5. Using Foursquare API to explore neighborhoods

In [10]:
# Foursquare Credentials
CLIENT_ID = 'ECEXV2NTBJMYXKUZGNLTSFMS0L5SUTFKJ34D2VP3PVJXHS2X' # your Foursquare ID
CLIENT_SECRET = 'P0L3D33OR0KZZJDFMTPPGAS4O411DOHOVN4YWGDSL135PF2A' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ECEXV2NTBJMYXKUZGNLTSFMS0L5SUTFKJ34D2VP3PVJXHS2X
CLIENT_SECRET:P0L3D33OR0KZZJDFMTPPGAS4O411DOHOVN4YWGDSL135PF2A


In [11]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT) 
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
boston_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Allston 
Back Bay 
Bay Village
Beacon Hill
Brighton
Charlestown
Chinatown – Leather District
Dorchester
Downtown
East Boston 
Fenway-Kenmore
Hyde Park
Jamaica Plain
Mattapan
Mid-Dorchester
Mission Hill
North End 
Roslindale 
Roxbury 
South Boston
South End 
West End 
West Roxbury 


### Checking the Size of the Resulting Dataframe


In [13]:
print(boston_venues.shape)
boston_venues.head()

(1185, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Allston,42.293046,-71.067169,Daily Table,42.295689,-71.071979,Market
1,Allston,42.293046,-71.067169,SUBWAY,42.291964,-71.071746,Sandwich Place
2,Allston,42.293046,-71.067169,Citizens Bank,42.291754,-71.072055,Bank
3,Allston,42.293046,-71.067169,McDonald's,42.290421,-71.071777,Fast Food Restaurant
4,Allston,42.293046,-71.067169,Walgreens,42.291119,-71.071810,Pharmacy


### Number of Venues Returned for Each Neighborhood

In [14]:
boston_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allston,9,9,9,9,9,9
Back Bay,100,100,100,100,100,100
Bay Village,38,38,38,38,38,38
Beacon Hill,36,36,36,36,36,36
Brighton,29,29,29,29,29,29
Charlestown,51,51,51,51,51,51
Chinatown – Leather District,100,100,100,100,100,100
Dorchester,100,100,100,100,100,100
Downtown,67,67,67,67,67,67


### Number of Unique Categories That Can Be Curated From all the Returned Venues

In [15]:
print('There are {} uniques categories.'.format(len(boston_venues['Venue Category'].unique())))

There are 195 uniques categories.


In [16]:
# list of unique categories
boston_venues['Venue Category'].unique()[:25]

array(['Market', 'Sandwich Place', 'Bank', 'Fast Food Restaurant',
       'Pharmacy', 'Fried Chicken Joint', 'Pizza Place',
       'Caribbean Restaurant', 'Park', 'Wine Shop', 'Miscellaneous Shop',
       'Bookstore', 'Furniture / Home Store', 'New American Restaurant',
       'Clothing Store', 'Social Club', 'Comic Shop',
       'Comfort Food Restaurant', 'Wine Bar', 'Sushi Restaurant',
       'Department Store', 'Hotel', 'Ramen Restaurant',
       'Gym / Fitness Center', 'Bar'], dtype=object)

In [17]:
# check if results contain 'Movie Theater'
'Movie Theater' in boston_venues['Venue Category'].unique()

True

# 6. Analyzing Each Neighborhood

In [18]:
# one hot encoding
boston_onehot = pd.get_dummies(boston_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
boston_onehot['Neighborhood'] = boston_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [boston_onehot.columns[-1]] + list(boston_onehot.columns[:-1])
boston_onehot = boston_onehot[fixed_columns]

print(boston_onehot.shape)
boston_onehot.head()

(1185, 195)


,Zoo Exhibit,Accessories Store,African Restaurant,American Restaurant,Arepa Restaurant,Art Gallery,Art Museum,Asian Restaurant,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bed & Breakfast,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Stop,Café,Cambodian Restaurant,Caribbean Restaurant,Chinese Restaurant,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Electronics Store,Ethiopian Restaurant,Event Service,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hockey Arena,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Liquor Store,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Resort,Restaurant,River,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Skating Rink,Ski Chalet,Snack Place,Soccer Field,Social Club,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Tour Provider,Tourist Information Center,Track,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,Allston,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Allston,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Allston,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category



In [19]:
boston_grouped = boston_onehot.groupby('Neighborhood').mean().reset_index()
print(boston_grouped.shape)
boston_grouped

(23, 195)


,Neighborhood,Zoo Exhibit,Accessories Store,African Restaurant,American Restaurant,Arepa Restaurant,Art Gallery,Art Museum,Asian Restaurant,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bed & Breakfast,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Stop,Café,Cambodian Restaurant,Caribbean Restaurant,Chinese Restaurant,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Electronics Store,Ethiopian Restaurant,Event Service,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hockey Arena,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Liquor Store,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Resort,Restaurant,River,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Skating Rink,Ski Chalet,Snack Place,Soccer Field,Social Club,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Tour Provider,Tourist Information Center,Track,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Allston,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.111111,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.111

In [20]:
len(boston_grouped[boston_grouped["Movie Theater"] > 0])

5

### Printing each neighborhood along with the top 5 most common venues

In [21]:
num_top_venues = 5

for hood in boston_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = boston_grouped[boston_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allston ----
                  venue  freq
0   Fried Chicken Joint  0.11
1  Caribbean Restaurant  0.11
2  Fast Food Restaurant  0.11
3        Sandwich Place  0.11
4           Pizza Place  0.11


----Back Bay ----
            venue  freq
0  Clothing Store  0.09
1     Coffee Shop  0.04
2  Sandwich Place  0.04
3           Hotel  0.03
4   Women's Store  0.03


----Bay Village----
               venue  freq
0            Theater  0.13
1                Spa  0.08
2     Sandwich Place  0.08
3  French Restaurant  0.05
4             Bakery  0.05


----Beacon Hill----
                venue  freq
0         Pizza Place  0.08
1        Gourmet Shop  0.06
2  Italian Restaurant  0.06
3   French Restaurant  0.06
4        Dessert Shop  0.03


----Brighton----
               venue  freq
0        Yoga Studio  0.07
1        Coffee Shop  0.07
2        Pizza Place  0.07
3    Bubble Tea Shop  0.03
4  College Cafeteria  0.03


----Charlestown----
         venue  freq
0  Pizza Place  0.10
1        Hotel  0.08

### Top 10 Venues for Each Neighborhood

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = boston_grouped['Neighborhood']

for ind in np.arange(boston_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(boston_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allston,Bank,Sandwich Place,Fried Chicken Joint,Market,Caribbean Restaurant,Park,Fast Food Restaurant,Pharmacy,Pizza Place,Dessert Shop
1,Back Bay,Clothing Store,Sandwich Place,Coffee Shop,Ice Cream Shop,Accessories Store,Bookstore,Grocery Store,Hotel,Women's Store,Italian Restaurant
2,Bay Village,Theater,Sandwich Place,Spa,Bakery,Performing Arts Venue,French Restaurant,Deli / Bodega,Rental Car Location,Dive Bar,Donut Shop
3,Beacon Hill,Pizza Place,Gourmet Shop,Italian Restaurant,French Restaurant,Ice Cream Shop,Lake,Cycle Studio,Dessert Shop,Park,Outdoor Sculpture
4,Brighton,Yoga Studio,Pizza Place,Coffee Shop,Thai Restaurant,Bubble Tea Shop,Snack Place,Rock Club,Food Court,Music Venue,Chinese Restaurant
5,Charlestown,Pizza Place,Hotel,Bar,Donut Shop,Park,Sports Bar,Sandwich Place,Convenience Store,Sporting Goods Shop,Church
6,Chinatown – Leather District,Chinese Restaurant,Bakery,Asian Restaurant,Sandwich Place,Coffee Shop,Café,Sushi Restaurant,Theater,Park,Bubble Tea Shop
7,Dorchester,Coffee Shop,Italian Restaurant,Bakery,French Restaurant,Food Truck,American Restaurant,Sandwich Place,Asian Restaurant,Hotel,Mexican Restaurant
8,Downtown,Hotel,Park,American Restaurant,Sandwich Place,Historic Site,Coffee Shop,Italian Restaurant,Deli / Bodega,Greek Restaurant,Seafood Restaurant
9,East Boston,Bakery,Chinese Restaurant,Sandwich Place,Coffee Shop,American Restaurant,Asian Restaurant,Sushi Restaurant,Food Truck,Mexican Restaurant,Dessert Shop


### Is Movie Theater a Top 10 venue in any Neighborhood?

In [23]:
print('Movie Theater' in neighborhoods_venues_sorted)

False


### Creating a new DataFrame for Movie Theater Data Only

In [24]:
boston_movie = boston_grouped[["Neighborhood", "Movie Theater"]]
boston_movie.head()

,Neighborhood,Movie Theater
0,Allston,0.0
1,Back Bay,0.0
2,Bay Village,0.0
3,Beacon Hill,0.0
4,Brighton,0.0


# 7. Cluster Neighborhoods

##### Using k-means to cluster neighborhoods of Boston into 3 clusters

In [25]:
# setting number of clusters
kclusters = 3

boston_clustering = boston_movie.drop(["Neighborhood"], 1)

# running k-means
k = KMeans(n_clusters = kclusters, random_state = 0).fit(boston_clustering)

# checking the cluster labels generated for each row in dataframe
k.labels_[:]

array([0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0], dtype=int32)

In [26]:
# create a new dataframe including the cluster and top 10 venues for each neighborhood
b_movie_merged = boston_movie.copy()

# add clustering labels
b_movie_merged["Cluster Label"] = k.labels_
b_movie_merged

,Neighborhood,Movie Theater,Cluster Label
0,Allston,0.000000,0
1,Back Bay,0.000000,0
2,Bay Village,0.000000,0
3,Beacon Hill,0.000000,0
4,Brighton,0.000000,0
5,Charlestown,0.000000,0
6,Chinatown – Leather District,0.010000,1
7,Dorchester,0.010000,1
8,Downtown,0.000000,0
9,East Boston,0.000000,0


In [27]:
# merge to add latitude/longitude for each neighborhood
b_movie_merged = b_movie_merged.join(df.set_index("Neighborhood"), on="Neighborhood")

print(b_movie_merged.shape)
b_movie_merged.head()

(23, 5)


,Neighborhood,Movie Theater,Cluster Label,Latitude,Longitude
0,Allston,0.0,0,42.293046,-71.067169
1,Back Bay,0.0,0,42.349990,-71.087650
2,Bay Village,0.0,0,42.348168,-71.068471
3,Beacon Hill,0.0,0,42.358420,-71.068600
4,Brighton,0.0,0,42.352131,-71.124924


In [28]:
# sorting results by Cluster Label
print(b_movie_merged.shape)
b_movie_merged.sort_values(["Cluster Label"], inplace=True)
b_movie_merged

(23, 5)


,Neighborhood,Movie Theater,Cluster Label,Latitude,Longitude
0,Allston,0.000000,0,42.293046,-71.067169
20,South End,0.000000,0,42.342560,-71.073580
19,South Boston,0.000000,0,42.352250,-71.055691
18,Roxbury,0.000000,0,42.330294,-71.089499
17,Roslindale,0.000000,0,42.281813,-71.137111
16,North End,0.000000,0,42.365490,-71.052970
15,Mission Hill,0.000000,0,42.335780,-71.109810
13,Mattapan,0.000000,0,42.278225,-71.096083
12,Jamaica Plain,0.000000,0,42.305850,-71.119095
11,Hyde Park,0.000000,0,42.274864,-71.119856


### Visualizing the resulting clusters

In [29]:
# create map
m_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(b_movie_merged['Latitude'], b_movie_merged['Longitude'], b_movie_merged['Neighborhood'], b_movie_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(m_clusters)
       
m_clusters

# 8. Examing the Clusters

### Cluster 0

In [30]:
b_movie_merged.loc[b_movie_merged['Cluster Label'] == 0]

,Neighborhood,Movie Theater,Cluster Label,Latitude,Longitude
0,Allston,0.0,0,42.293046,-71.067169
20,South End,0.0,0,42.342560,-71.073580
19,South Boston,0.0,0,42.352250,-71.055691
18,Roxbury,0.0,0,42.330294,-71.089499
17,Roslindale,0.0,0,42.281813,-71.137111
16,North End,0.0,0,42.365490,-71.052970
15,Mission Hill,0.0,0,42.335780,-71.109810
13,Mattapan,0.0,0,42.278225,-71.096083
12,Jamaica Plain,0.0,0,42.305850,-71.119095
11,Hyde Park,0.0,0,42.274864,-71.119856


### Cluster 1

In [31]:
b_movie_merged.loc[b_movie_merged['Cluster Label'] == 1]

,Neighborhood,Movie Theater,Cluster Label,Latitude,Longitude
21,West End,0.010753,1,42.363940,-71.06739
7,Dorchester,0.010000,1,42.351349,-71.05285
14,Mid-Dorchester,0.010000,1,42.351349,-71.05285
6,Chinatown – Leather District,0.010000,1,42.352510,-71.06090


### Cluster 2

In [32]:
b_movie_merged.loc[b_movie_merged['Cluster Label'] == 2]

,Neighborhood,Movie Theater,Cluster Label,Latitude,Longitude
10,Fenway-Kenmore,0.018519,2,42.34355,-71.10157


### Movie Theater Venues

In [33]:
array = ['Movie Theater']
boston_venues.loc[boston_venues['Venue Category'].isin(array)]
    

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
300,Chinatown – Leather District,42.352510,-71.06090,AMC Loews Boston Common 19,42.353308,-71.063974,Movie Theater
413,Dorchester,42.351349,-71.05285,Kerasotes Showplace Icon Theater - Boston,42.352848,-71.047335,Movie Theater
651,Fenway-Kenmore,42.343550,-71.10157,Regal Fenway & RPX,42.344146,-71.101504,Movie Theater
765,Mid-Dorchester,42.351349,-71.05285,Kerasotes Showplace Icon Theater - Boston,42.352848,-71.047335,Movie Theater
1135,West End,42.363940,-71.06739,Mugar Omni IMAX Theatre,42.367074,-71.070081,Movie Theater


# Findings

### The concentration of movie theaters is in descending order among the clusters from Cluster 2 down to Cluster 0. The Fenway-Kenmore neighborhood is the most concentrated movie theater location out of all the Boston neighborhoods, as well as the only neighborhood in Cluster 2. Cluster 1 contains four neighborhoods that all have at least one theater. Cluster 0 contains 18 neighborhoods that all have no movie theaters. My recommendation would be to open up a movie theater in Cluster 0. Any movie theater opening up in Allston, South End, South Boston, Roxbury, Roslindale, North End, Mission Hill, Mattapan, Jamaica Plain, Hyde Park, Downtown, Charlestown, Brighton, Beacon Hill, Bay Village, Back Bay, East Boston, or  West Roxbury would face no competition. Overall, Boston has very few movie theaters. Opening up a theater in Cluster 0 would be a great opportunity for any property developer to take on.